In [ ]:
import os
import pandas as pd
import pyodbc

server = 'juanlopezch.database.windows.net'
database = 'Spotify_charts'
username = '*********'
password = '***********'

carpeta_principal = r'C:****Projects\Spotify 2024\Spotify Charts'

Spotify_charts = []

In [ ]:
# After downloading several weekly files, all of them are added to one single df
for filename in os.listdir(carpeta_principal):
    if filename.endswith(".csv"):
        filepath = os.path.join(carpeta_principal, filename)
        df = pd.read_csv(filepath)
        df['end_of_week'] = filename

        Spotify_charts.append(df)

In [ ]:
Spotify_charts = pd.concat(Spotify_charts, ignore_index=True)

In [ ]:
# Extracting the date from the file name and formating it.
Spotify_charts['end_of_week'] = Spotify_charts['end_of_week'].str.replace('.csv', '')
Spotify_charts['end_of_week'] = Spotify_charts['end_of_week'].str.replace('regional-global-weekly-', '')
Spotify_charts['end_of_week'] = pd.to_datetime(Spotify_charts['end_of_week'], format='%Y-%m-%d', errors = 'coerce')

In [ ]:
# Created a Unique ID for each song-week
Spotify_charts["UniqueID"] = Spotify_charts.apply(lambda x: x["uri"].split(":")[-1] + 
                                      str(x["rank"]) + 
                                      str(x["end_of_week"].year)[2:] + 
                                      str(x["end_of_week"].month).zfill(2) +
                                      str(x["end_of_week"].day).zfill(2), axis=1)

In [ ]:
# Removing apostrophes from strings.
Spotify_charts['track_name'] = Spotify_charts['track_name'].str.replace(r"'", '')
Spotify_charts['artist_names'] = Spotify_charts['artist_names'].str.replace(r"'", '')
Spotify_charts['source'] = Spotify_charts['source'].str.replace(r"'", '')

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
    
   
 #Inserting Dataframe into SQL Server:
for index, row in result.iterrows():
     cursor.execute("INSERT INTO dbo.WeeklyCharts (UniqueID,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,end_of_week) values(?,?,?,?,?,?,?,?,?,?,?)", 
                    row['UniqueID'], row['rank'], row['uri'], row['artist_names'], row['track_name'], row['source'], row['peak_rank'], row['previous_rank'], row['weeks_on_chart'], row['streams'], row['end_of_week'])
     cnxn.commit()


cursor.close()